In [24]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import gym
from collections import deque
import random
import time
import yaml
from klqr import *
from train_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
with open('config.yml','r') as ymlfile:
    config = yaml.load(ymlfile)
    
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
env = gym.make('Pendulum-v0')
agent = klqr(config,sess)
agent.build_model()
train_agent(agent,env,max_timesteps=40000, n_transitions_between_updates=100,
                n_optim_steps_per_update=200,
                n_iters_per_p_update=5,
                )

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
z shape: (?, 3)
********** Iteration 1 ************
	Episode Len: 0
	Last Episode Cost: 0.000000 vs optimal 0.000000
********** Iteration 2 ************
	Episode Len: 200
	Last Episode Cost: 1130.590261 vs optimal 0.000000
********** Iteration 3 ************
	Episode Len: 200
	Last Episode Cost: 1130.590261 vs optimal 0.000000
********** Iteration 4 ************
	Episode Len: 200
	Last Episode Cost: 1215.490095 vs optimal 0.000000
********** Iteration 5 ************
[[ 9.6599627e-01 -3.2687888e-02  7.6765503e-04]
 [-3.2687888e-02  9.5449966e-01  7.0713223e-03]
 [ 7.6765503e-04  7.0713223e-03  9.3384463e-01]]
	Episode Len: 200
	Last Episode Cost: 1215.490095 vs optimal 0.000000
********** Iteration 6 ************
	Episode Len: 200
	Last Episode Cost: 895.214727 vs optimal 0.000000
********** 

In [30]:
#test

agent.max_a = 100.0
agent.min_a = -100.0
train_agent(agent,env,max_timesteps=1000,training=False)

********** Iteration 1 ************
	Episode Len: 0
	Last Episode Cost: 0.000000 vs optimal 0.000000
********** Iteration 2 ************
	Episode Len: 200
	Last Episode Cost: 788.392811 vs optimal 0.000000
********** Iteration 3 ************
	Episode Len: 200
	Last Episode Cost: 788.392811 vs optimal 0.000000
********** Iteration 4 ************
	Episode Len: 200
	Last Episode Cost: 852.464238 vs optimal 0.000000
********** Iteration 5 ************
	Episode Len: 200
	Last Episode Cost: 852.464238 vs optimal 0.000000
********** Iteration 6 ************
	Episode Len: 200
	Last Episode Cost: 892.304644 vs optimal 0.000000
********** Iteration 7 ************
	Episode Len: 200
	Last Episode Cost: 892.304644 vs optimal 0.000000
********** Iteration 8 ************
	Episode Len: 200
	Last Episode Cost: 907.518440 vs optimal 0.000000
********** Iteration 9 ************
	Episode Len: 200
	Last Episode Cost: 907.518440 vs optimal 0.000000
********** Iteration 10 ************
	Episode Len: 200
	Las

In [13]:
#test

agent.max_a = 100.0
agent.min_a = -100.0
train_agent(agent,env,max_timesteps=1000,training=False)

********** Iteration 1 ************
	Episode Len: 100
	Last Episode Cost: 15.114967 vs optimal 0.000000
********** Iteration 2 ************
	Episode Len: 100
	Last Episode Cost: 2.847836 vs optimal 2.355781
********** Iteration 3 ************
	Episode Len: 100
	Last Episode Cost: 12.386197 vs optimal 11.864259
********** Iteration 4 ************
	Episode Len: 100
	Last Episode Cost: 13.126852 vs optimal 11.716004
********** Iteration 5 ************
	Episode Len: 100
	Last Episode Cost: 8.160144 vs optimal 7.246125
********** Iteration 6 ************
	Episode Len: 100
	Last Episode Cost: 12.285117 vs optimal 11.117412
********** Iteration 7 ************
	Episode Len: 100
	Last Episode Cost: 8.954077 vs optimal 7.739204
********** Iteration 8 ************
	Episode Len: 100
	Last Episode Cost: 14.499120 vs optimal 13.201736
********** Iteration 9 ************
	Episode Len: 100
	Last Episode Cost: 5.719255 vs optimal 5.130324
********** Iteration 10 ************
	Episode Len: 100
	Last Epi

In [36]:
sess.run(agent.A)

array([[0.792256  , 0.09586458, 0.00956542],
       [0.09225597, 0.79586446, 0.09456539],
       [0.00725596, 0.09586458, 0.7945655 ]], dtype=float32)

In [37]:
sess.run(agent.B)

array([[ 1.0015329 , -0.00488705,  0.00397779],
       [ 0.00153268,  0.9951129 ,  0.00397778],
       [ 0.00153271, -0.0048871 ,  1.0039778 ]], dtype=float32)

In [38]:
sess.run(agent.Q)

array([[ 0.7462247 ,  0.09592342, -0.02262163],
       [ 0.09592342,  0.6999506 ,  0.14456768],
       [-0.02262163,  0.14456768,  0.77230483]], dtype=float32)

In [39]:
sess.run(agent.R)

array([[ 9.958884  , -1.0673071 , -0.41716802],
       [-1.0673071 , 10.604416  , -2.067732  ],
       [-0.41716802, -2.067732  , 11.993966  ]], dtype=float32)

In [40]:
A,B,Q,R = (env.env.A, env.env.B, env.env.Q, env.env.R)
P = Q
for k in range(100):
    P = Q + A.T @ P @ A - A.T @ P.T @ B @ np.linalg.inv(R + B.T @ P @ B ) @ B.T @ P @ A
    P = 0.5*(P + P.T)

In [41]:
P

array([[2.23999771, 0.54694411, 0.19286546],
       [0.54694411, 2.35082156, 0.54694411],
       [0.19286546, 0.54694411, 2.23999771]])

In [42]:
sess.run(agent.P)

array([[1.7117685 , 0.55636847, 0.07273098],
       [0.55636847, 1.7515693 , 0.6495986 ],
       [0.07273098, 0.6495986 , 1.8076872 ]], dtype=float32)